In [ ]:
import csv  
import re


# Data Cleaning:
Pour télécharger les datas pour ce notebook, vous pouvez utilisez le [script associé](/nautilus_nlp/data/external/get_stanfordtweets.sh)

In [3]:

train = open('../data/processed/tweets.train','w',encoding='utf-8')  
test = open('../data/processed/tweets.valid','w',encoding='utf-8')  
with open('../data/external/tweets_sentiment/training.1600000.processed.noemoticon.csv', mode='r', encoding = "ISO-8859-1") as csv_file:  
    csv_reader = csv.DictReader(csv_file, fieldnames=['target', 'id', 'date', 'flag', 'user', 'text'])
    line = 0
    for row in csv_reader:
        # Clean the training data
        # First we lower case the text
        text = row["text"].lower()
        # remove links
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
        #Remove usernames
        text = re.sub('@[^\s]+','', text)
        # replace hashtags by just words
        text = re.sub(r'#([^\s]+)', r'\1', text)
        #correct all multiple white spaces to a single white space
        text = re.sub('[\s]+', ' ', text)
        # Additional clean up : removing words less than 3 chars, and remove space at the beginning and teh end
        text = re.sub(r'\W*\b\w{1,3}\b', '', text)
        text = text.strip()
        line = line + 1
        # Split data into train and validation
        if line%16 == 0:
            print(f'__label__{row["target"]} {text}', file=test)
        else:
            print(f'__label__{row["target"]} {text}', file=train)

# Model Training

In [2]:
from nautilus_nlp.models import Fasttext_classifier 

In [3]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
    
train_data = '../data/processed/tweets.train'
valid_data = '../data/processed/tweets.valid'

model = Fasttext_classifier.Fasttext_clf()


In [8]:
print_results(*model.test(valid_data))


N	100000
P@1	0.759
R@1	0.759


In [6]:
%%timeit
model.train(train_data)


12.9 s ± 1.26 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Il faut donc 13 secondes pour entrainer un modèle sur 1.5 Millions de tweets, avec une précision de 0.759. Not bad

In [9]:
model.save_model('/home/nautilus_nlp/models/sentiments.bin')

In [11]:
!ls -lh '/home/nautilus_nlp/models/sentiments.bin'

-rw-r--r-- 1 root root 232M Mar 14 12:51 /home/nautilus_nlp/models/sentiments.bin


Ce modèle fait environ 230Mo

Maintenant si on quantize ce modèle

In [12]:
model.quantize(input=train_data, qnorm=True, retrain=True, cutoff=100000)
print_results(*model.test(valid_data))
model.save_model('/home/nautilus_nlp/models/sentiments.ftz')

N	100000
P@1	0.759
R@1	0.759


In [13]:
!ls -lh '/home/nautilus_nlp/models/sentiments.ftz'

-rw-r--r-- 1 root root 6.8M Mar 14 12:53 /home/nautilus_nlp/models/sentiments.ftz
